In [ ]:
!sudo apt update

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:7 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [969 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,59

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
!pip install -q findspark

In [ ]:
import findspark
findspark.init()

In [7]:
from pyspark.context import SparkContext
#create SparkContext
sc = SparkContext("local", "My First Spark Application")
print("SparkContext :",sc)

SparkContext : <SparkContext master=local appName=My First Spark Application>


In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [9]:
from pyspark.sql.functions import *

In [77]:
dfRatings = spark.read.option("inferSchema", "true").option("delimiter", "\t").csv('/content/u.data')
dfRatings = dfRatings.withColumnRenamed("_c0", 'userId').withColumnRenamed("_c1", "movieId").withColumnRenamed("_c2", "rating").withColumnRenamed("_c3","timeStamp")
dfRatings.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timeStamp|
+------+-------+------+---------+
|   196|    242|     3|881250949|
|   186|    302|     3|891717742|
|    22|    377|     1|878887116|
|   244|     51|     2|880606923|
|   166|    346|     1|886397596|
|   298|    474|     4|884182806|
|   115|    265|     2|881171488|
|   253|    465|     5|891628467|
|   305|    451|     3|886324817|
|     6|     86|     3|883603013|
+------+-------+------+---------+
only showing top 10 rows



In [78]:
dfUsers = spark.read.option("inferSchema", "true").option("delimiter", "|").csv('/content/u.user')
dfUsers =dfUsers.withColumnRenamed("_c0","userId").withColumnRenamed("_c1", "age").withColumnRenamed("_c2", "gender").withColumnRenamed("_c3","occupation").withColumnRenamed("_c4","zipcode")
dfUsers.show(10)

+------+---+------+-------------+-------+
|userId|age|gender|   occupation|zipcode|
+------+---+------+-------------+-------+
|     1| 24|     M|   technician|  85711|
|     2| 53|     F|        other|  94043|
|     3| 23|     M|       writer|  32067|
|     4| 24|     M|   technician|  43537|
|     5| 33|     F|        other|  15213|
|     6| 42|     M|    executive|  98101|
|     7| 57|     M|administrator|  91344|
|     8| 36|     M|administrator|  05201|
|     9| 29|     M|      student|  01002|
|    10| 53|     M|       lawyer|  90703|
+------+---+------+-------------+-------+
only showing top 10 rows



In [79]:
dfMovies = spark.read.option("inferSchema", "true").option("delimiter", "|").csv('/content/u.item')
dfMovies = dfMovies.withColumnRenamed("_c0", "movieId").withColumnRenamed("_c1", "movie title").withColumnRenamed("_c2", "release date").withColumnRenamed("_c3","video relase date").withColumnRenamed("_c4", "IMDb URL")
dfMovies.show(10)

+-------+--------------------+------------+-----------------+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|movieId|         movie title|release date|video relase date|            IMDb URL|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|
+-------+--------------------+------------+-----------------+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|      1|    Toy Story (1995)| 01-Jan-1995|             null|http://us.imdb.co...|  0|  0|  0|  1|  1|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
|      2|    GoldenEye (1995)| 01-Jan-1995|             null|http://us.imdb.co...|  0|  1|  1|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|      3|   Four Rooms (1995)| 01-Jan-1995|             null|http://us.imdb.co...|  0|  0|  0|  0|  0|   0|   0|   0|   0|   0|   0|   

In [80]:
#cau 2
dfRatings.groupBy("userId").agg(count("rating"), max('rating'), min('rating'), mean('rating')).show()

+------+-------------+-----------+-----------+------------------+
|userId|count(rating)|max(rating)|min(rating)|       avg(rating)|
+------+-------------+-----------+-----------+------------------+
|   148|           65|          5|          1|               4.0|
|   463|          133|          5|          1|2.8646616541353382|
|   471|           31|          5|          1|3.3870967741935485|
|   496|          129|          5|          1|3.0310077519379846|
|   833|          267|          5|          1| 3.056179775280899|
|   243|           81|          5|          1|3.6419753086419755|
|   392|          111|          5|          1| 4.045045045045045|
|   540|           63|          5|          2|3.7142857142857144|
|   623|           45|          5|          1|3.7333333333333334|
|   737|           33|          5|          1|3.9696969696969697|
|   858|           21|          5|          1|3.4285714285714284|
|   897|          185|          5|          1| 3.962162162162162|
|    31|  

In [81]:
#cau 3
dfRatings.filter("userId=1").select(collect_list("movieId")).collect()

[Row(collect_list(movieId)=[61, 189, 33, 160, 20, 202, 171, 265, 155, 117, 47, 222, 253, 113, 227, 17, 90, 64, 92, 228, 266, 121, 114, 132, 74, 134, 98, 186, 221, 84, 31, 70, 60, 177, 27, 260, 145, 174, 159, 82, 56, 272, 80, 229, 140, 225, 235, 120, 125, 215, 6, 104, 49, 206, 76, 72, 185, 96, 213, 233, 258, 81, 78, 212, 143, 151, 51, 175, 107, 218, 209, 259, 108, 262, 12, 14, 97, 44, 53, 163, 210, 184, 157, 201, 150, 183, 248, 208, 128, 242, 148, 112, 193, 264, 219, 232, 236, 252, 200, 180, 250, 85, 91, 10, 254, 129, 241, 130, 255, 103, 118, 54, 267, 24, 86, 196, 39, 164, 230, 36, 23, 224, 73, 67, 65, 190, 100, 226, 243, 154, 214, 161, 62, 188, 102, 69, 170, 38, 9, 246, 22, 21, 179, 187, 135, 68, 146, 176, 166, 138, 247, 89, 2, 30, 63, 249, 269, 32, 141, 211, 40, 270, 133, 239, 194, 256, 220, 93, 8, 205, 234, 105, 147, 99, 1, 197, 173, 75, 268, 34, 144, 271, 119, 26, 158, 37, 181, 136, 257, 237, 131, 109, 182, 71, 223, 46, 169, 41, 162, 110, 66, 77, 199, 57, 50, 192, 178, 5, 87, 238, 1

In [82]:
#cau 4
dfMovies.filter('movieId = 1').show()

+-------+----------------+------------+-----------------+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|movieId|     movie title|release date|video relase date|            IMDb URL|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|
+-------+----------------+------------+-----------------+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|      1|Toy Story (1995)| 01-Jan-1995|             null|http://us.imdb.co...|  0|  0|  0|  1|  1|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
+-------+----------------+------------+-----------------+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+



In [83]:
#cau 5 
dfMovies.filter(col("movie title")=="Taxi Driver (1976)").show()

+-------+------------------+------------+-----------------+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|movieId|       movie title|release date|video relase date|            IMDb URL|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|
+-------+------------------+------------+-----------------+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|     23|Taxi Driver (1976)| 16-Feb-1996|             null|http://us.imdb.co...|  0|  0|  0|  0|  0|   0|   0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
+-------+------------------+------------+-----------------+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+



In [84]:
c5 = dfRatings.groupBy("movieId").agg(count("rating") ,max('rating'), min('rating'), mean('rating'))
c5.filter(col("movieId") == 23).show()

+-------+-------------+-----------+-----------+------------------+
|movieId|count(rating)|max(rating)|min(rating)|       avg(rating)|
+-------+-------------+-----------+-----------+------------------+
|     23|          182|          5|          2|4.1208791208791204|
+-------+-------------+-----------+-----------+------------------+



In [85]:
#c6
df_movie = dfRatings.groupBy("movieId").agg(count("rating") ,max('rating'), min('rating'), mean('rating'))
df_movie.show()

+-------+-------------+-----------+-----------+------------------+
|movieId|count(rating)|max(rating)|min(rating)|       avg(rating)|
+-------+-------------+-----------+-----------+------------------+
|    496|          231|          5|          1| 4.121212121212121|
|    471|          221|          5|          1|3.6108597285067874|
|    463|           71|          5|          1| 3.859154929577465|
|    148|          128|          5|          1|          3.203125|
|   1342|            2|          4|          1|               2.5|
|    833|           49|          5|          1| 3.204081632653061|
|   1088|           13|          4|          1| 2.230769230769231|
|   1591|            6|          5|          1|3.1666666666666665|
|   1238|            8|          5|          2|             3.125|
|   1580|            1|          1|          1|               1.0|
|   1645|            1|          4|          4|               4.0|
|    392|           68|          5|          1|3.5441176470588

In [86]:
#cau 7
df_group = df_movie.select(col('movieId'), col('count(rating)')).join(dfMovies.select(col("movieId"), col("movie title")), df_movie.movieId==dfMovies.movieId, 'inner')

In [87]:
df_group.sort(desc("count(rating)")).select(col("movie title")).limit(10).show()

+--------------------+
|         movie title|
+--------------------+
|    Star Wars (1977)|
|      Contact (1997)|
|        Fargo (1996)|
|Return of the Jed...|
|    Liar Liar (1997)|
|English Patient, ...|
|       Scream (1996)|
|    Toy Story (1995)|
|Air Force One (1997)|
|Independence Day ...|
+--------------------+



In [88]:
# cau 8
df_group.filter(col('count(rating)') < 10).show()

+-------+-------------+-------+--------------------+
|movieId|count(rating)|movieId|         movie title|
+-------+-------------+-------+--------------------+
|   1342|            2|   1342|Convent, The (Con...|
|   1591|            6|   1591|Duoluo tianshi (1...|
|   1238|            8|   1238|   Full Speed (1996)|
|   1580|            1|   1580|     Liebelei (1933)|
|   1645|            1|   1645|Butcher Boy, The ...|
|    858|            3|    858|Amityville: Dollh...|
|   1395|            6|   1395|Hurricane Streets...|
|   1522|            7|   1522|Trial by Jury (1994)|
|   1507|            1|   1507|Three Lives and O...|
|    897|            2|    897| Time Tracers (1995)|
|   1460|            1|   1460|    Sleepover (1995)|
|   1322|            6|   1322|Metisse (Caf� au ...|
|   1270|            7|   1270|Life with Mikey (...|
|   1339|            1|   1339|Stefano Quantesto...|
|   1650|            1|   1650|Butcher Boy, The ...|
|   1618|            1|   1618|King of New Yor

In [89]:
# cau 9 
df_group.agg(max("count(rating)")).show()

+------------------+
|max(count(rating))|
+------------------+
|               583|
+------------------+



In [90]:
df_group.filter(col("count(rating)") == 583).select(col("movie title")).show()

+----------------+
|     movie title|
+----------------+
|Star Wars (1977)|
+----------------+



In [91]:
#cau 10
dfrt = dfRatings.select(col("userId"), col("movieId").alias("movieId_r"), col("rating"))
dfmv = dfMovies.withColumnRenamed('movieId', 'movieId_m')
df_gr = dfrt.join(dfmv, dfrt.movieId_r==dfmv.movieId_m, "inner")
df_gr1 = df_gr.select(col("movieId_m"),col("rating"), *[col(x) for x in df_gr.columns[8:]])

In [71]:
df_gr1.show(5)

+---------+------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|movieId_m|rating|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|
+---------+------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|      242|     3|  0|  0|  0|  0|  0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
|      302|     3|  0|  0|  0|  0|  0|   0|   1|   0|   0|   0|   1|   0|   0|   1|   0|   0|   1|   0|   0|
|      377|     1|  0|  0|  0|  0|  1|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
|       51|     2|  0|  0|  0|  0|  0|   0|   0|   0|   1|   0|   0|   0|   0|   0|   1|   0|   0|   1|   1|
|      346|     1|  0|  0|  0|  0|  0|   0|   1|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
+---------+------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
only showing top 5 

In [119]:
avg_max = 0
t = ''
for x in df_gr1.columns[2:]:
  avg_col = df_gr1.select('rating',col(x)).filter(col(x)!=0).agg(avg('rating')).collect()[0][0]
  
  if avg_max < avg_col:
    avg_max = avg_col
    t = x
t, avg_max

('_c15', 3.9215233698788228)

In [92]:
#cau 11
dfrt = dfRatings.select(col('userId'), col("movieId"), col('rating'))
dfus = dfUsers.select(col('userId'), col('occupation'))
df_n = dfrt.join(dfus, dfrt.userId == dfus.userId, "inner")
df_n1 = df_n.filter((df_n.occupation == "programmer") & (df_n.rating >= 4))

In [93]:
df_n2 = df_n1.select(col('movieId')).join(dfMovies, df_n1.movieId == dfMovies.movieId, "inner")
df_n3 = df_n2.select(*[col(x) for x in df_n2.columns[6:]])
df_nx = [collect_set(c).alias(c) for c in df_n3.columns]
result = df_n3.agg(*df_nx)
result.show()
# co ca 0 va 1 tuc là programmer da rating the loai do

+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|   _c5|   _c6|   _c7|   _c8|   _c9|  _c10|  _c11|  _c12|  _c13|  _c14|  _c15|  _c16|  _c17|  _c18|  _c19|  _c20|  _c21|  _c22|  _c23|
+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|[0, 1]|
+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+



In [122]:
#cau 12
df_x = dfRatings.select(col('userId'), col("movieId").alias("movieId_r"), col('rating'))
df_x1 = dfMovies.join(df_x , dfMovies.movieId==df_x.movieId_r, how = 'inner')

In [124]:
df_x1.filter((col("userId") == 1) & (col("rating").isNull())).show()

+-------+-----------+------------+-----------------+--------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+---------+------+
|movieId|movie title|release date|video relase date|IMDb URL|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|userId|movieId_r|rating|
+-------+-----------+------------+-----------------+--------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+---------+------+
+-------+-----------+------------+-----------------+--------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+---------+------+



In [96]:
#cau 13
df_rt4 = dfRatings.groupBy(col('movieId')).agg(min('rating').alias("min_rt")).filter(col('min_rt') >= 4)
df_rt4.show(5)

+-------+------+
|movieId|min_rt|
+-------+------+
|   1645|     4|
|   1650|     4|
|   1533|     4|
|   1653|     5|
|   1525|     4|
+-------+------+
only showing top 5 rows



In [97]:
# cau 16
df_m = dfRatings.select('userId','movieId').join(dfUsers.select('userId', 'occupation'), dfRatings.userId==dfUsers.userId, 'inner')
df_m=df_m.drop('userId')
df_m.show(5)

+-------+----------+
|movieId|occupation|
+-------+----------+
|    242|    writer|
|    302| executive|
|    377|    writer|
|     51|technician|
|    346|  educator|
+-------+----------+
only showing top 5 rows



In [98]:
df_m.groupBy('movieId', 'occupation').count().show()

+-------+-------------+-----+
|movieId|   occupation|count|
+-------+-------------+-----+
|    785|      student|   11|
|    603|    marketing|    7|
|    289|     educator|   31|
|    134|       doctor|    1|
|    756|entertainment|    3|
|    191|administrator|   21|
|    604|     educator|   18|
|    552|   programmer|    8|
|    671|      student|    4|
|    337|       writer|    4|
|    469|    executive|    3|
|    750|    librarian|    9|
|    155|    librarian|    7|
|    181|        other|   54|
|    781|      student|   24|
|     47|       artist|    3|
|    298|        other|   21|
|    498|    librarian|   12|
|    810|     engineer|    4|
|    823|administrator|    6|
+-------+-------------+-----+
only showing top 20 rows

